# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from nltk.tokenize import word_tokenize
from sklearn.model_selection import GridSearchCV

import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/bignollo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("Message", engine)
X = df['message']
Y = df.drop(["index","id","message","original","genre"], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text).lower()
    words = word_tokenize(text)
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
from sklearn.metrics import classification_report
y_pred = pd.DataFrame (y_pred, columns = y_test.columns)
for col in y_pred.columns:
    print(col + ":" + classification_report(y_test[col], y_pred[col]))

related:              precision    recall  f1-score   support

           0       0.70      0.25      0.37      2054
           1       0.80      0.96      0.88      6534
           2       0.58      0.39      0.47        64

    accuracy                           0.79      8652
   macro avg       0.69      0.54      0.57      8652
weighted avg       0.78      0.79      0.75      8652

request:              precision    recall  f1-score   support

           0       0.89      0.99      0.94      7180
           1       0.89      0.42      0.57      1472

    accuracy                           0.89      8652
   macro avg       0.89      0.71      0.76      8652
weighted avg       0.89      0.89      0.88      8652

offer:              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38

    accuracy                           1.00      8652
   macro avg       0.50      0.50      0.50      865

/opt/Anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


infrastructure_related:              precision    recall  f1-score   support

           0       0.93      1.00      0.97      8084
           1       0.00      0.00      0.00       568

    accuracy                           0.93      8652
   macro avg       0.47      0.50      0.48      8652
weighted avg       0.87      0.93      0.90      8652

transport:              precision    recall  f1-score   support

           0       0.96      1.00      0.98      8245
           1       0.71      0.12      0.20       407

    accuracy                           0.96      8652
   macro avg       0.84      0.56      0.59      8652
weighted avg       0.95      0.96      0.94      8652

buildings:              precision    recall  f1-score   support

           0       0.95      1.00      0.98      8211
           1       0.78      0.08      0.14       441

    accuracy                           0.95      8652
   macro avg       0.87      0.54      0.56      8652
weighted avg       0.94      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        #'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, )
cv.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_test = cv.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# try other model -> RandomForestClassifier
rf_model = pipeline(clf = AdaBoostClassifier|())
rf_model.fit(X_train, y_train)
y_pred_rf_test = rf_model.predict(X_test)

print(classification_report(y_test.values, y_pred_rf_test, target_names=y.columns.values))

### 9. Export your model as a pickle file

In [ ]:
# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(model_3, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.